In [64]:
import os
import pandas as pd
import glob
import pickle
from selenium import webdriver
from selenium.webdriver.common.by import By
import concurrent.futures 
from pycaret.classification import *
from sklearn.metrics import accuracy_score
import requests
from bs4 import BeautifulSoup
import time
import random
from fuzzywuzzy import fuzz, process
from os import walk
from datetime import datetime

In [65]:
def getname(name, l):
    if len(l) == 0:
        return ''
    highest = process.extractOne(name,l)
    return highest[0]

In [66]:
def winner(x):
    if x == 0:
        return 0
    elif x > 0:
        return 1
    else:
        return 2

In [67]:
def get_winner(url):
    page = requests.get(url)
    time.sleep(random.randint(0, 4))
    soup = BeautifulSoup(page.content, 'html.parser')
    score = soup.find_all("span", {"class": "css-bw7eig-topRow"})[0].text.split(' - ')
    score = int(score[0]) - int(score[1])
    score = winner(score)
    return score

In [70]:
filenames = next(walk('C:/Users\idotz/OneDrive - Northeastern University/Python/Soccer Analysis'), (None, None, []))[2]
filenames = [f for f in filenames if '2022' in f and '_data' in f and 'dict' not in f]

In [71]:
filenames

['20220826_data.pkl',
 '20220828_data.pkl',
 '20220829_data.pkl',
 '20220830_data.pkl',
 '20220831_data.pkl',
 '20220901_data.pkl',
 '20220902_data.pkl',
 '20220903_data.pkl',
 '20220904_data.pkl',
 '20220905_data.pkl',
 '20220906_data.pkl',
 '20220907_data.pkl',
 '20220908_data.pkl',
 '20220909_data.pkl',
 '20220910_data.pkl']

In [102]:
all_data = []
for f in filenames:
    open_file = open(f, "rb")
    d = pickle.load(open_file)
    all_data.append(d)
    open_file.close()
    
all_data = [i for j in all_data for i in j]

In [103]:
today = f"{datetime.now():%Y%m%d}"
open_file = open(f'{today}_datadict.pkl', "wb")
pickle.dump(all_data, open_file)
open_file.close()

In [88]:
f = []
for dic in all_data:
    d = {}
    for k, v in dic.items():
        try:
            d[k] = float(v[0]) - float(v[1])
        except:
            d[k] = v
    f.append(d)

In [98]:
#with odds
df = pd.DataFrame(all_data)
df = df[df['half_time'] == 'Half-Time']
df = df[(df['odds'] != '') & (df['odds'] != {})]
df = df.reset_index(drop = True)
df2 = pd.json_normalize(df['bet'])
df = pd.concat([df, df2], axis = 1)
df['o_names'] = df['odds_url'].apply(lambda x: ['draw'] + x.split('/')[-2].split('-v-'))
r = []
for i in df[['odds','o_names']].to_numpy():
    oo = [getname(name, list(i[0].keys())) for name in i[1]]
    oo = [i[0][key] for key in oo]
    r.append(oo)
df['oo'] = r
df['odds_predict'] = df['oo'].apply(lambda x: np.argmin(x))
split_df = pd.DataFrame(df['oo'].tolist(), columns=['0', '1', '2'])
df = pd.concat([df, split_df], axis=1)
df = df.drop(['match_name', 'half_time','odds_url','odds','o_names','oo','predicted_probs',
             'bet','draw','home','away','winner'], axis = 1)

In [79]:
scores = []
for i in range(0, len(df['url']), 4):
    x = list(df['url'][i:i + 4])
    print(x)
    with concurrent.futures.ThreadPoolExecutor() as executor:
        r = executor.map(get_winner, x)
    for result in r:
        scores.append(result)

['https://www.fotmob.com/match/3888550/matchfacts/puebla-vs-fc-juarez', 'https://www.fotmob.com/match/3888551/matchfacts/mazatlan-fc-vs-cf-america', 'https://www.fotmob.com/match/3900373/matchfacts/fc-utrecht-vs-ajax', 'https://www.fotmob.com/match/3904422/matchfacts/nantes-vs-toulouse']
['https://www.fotmob.com/match/3902828/matchfacts/dundee-united-vs-celtic', 'https://www.fotmob.com/match/3916965/matchfacts/gent-vs-royal-antwerp', 'https://www.fotmob.com/match/3887414/matchfacts/randers-fc-vs-aalborg-bk', 'https://www.fotmob.com/match/3887416/matchfacts/viborg-vs-lyngby']
['https://www.fotmob.com/match/3905446/matchfacts/lugano-vs-st.-gallen', 'https://www.fotmob.com/match/3900374/matchfacts/excelsior-vs-psv-eindhoven', 'https://www.fotmob.com/match/3900375/matchfacts/fc-volendam-vs-fc-twente', 'https://www.fotmob.com/match/3900963/matchfacts/aston-villa-vs-west-ham-united']
['https://www.fotmob.com/match/3900971/matchfacts/wolverhampton-wanderers-vs-newcastle-united', 'https://www.

['https://www.fotmob.com/match/3888623/matchfacts/necaxa-vs-leon', 'https://www.fotmob.com/match/3900986/matchfacts/everton-vs-liverpool', 'https://www.fotmob.com/match/3902849/matchfacts/celtic-vs-rangers', 'https://www.fotmob.com/match/3917967/matchfacts/mallorca-vs-girona']
['https://www.fotmob.com/match/3919120/matchfacts/fiorentina-vs-juventus', 'https://www.fotmob.com/match/3903581/matchfacts/bayer-leverkusen-vs-sc-freiburg', 'https://www.fotmob.com/match/3903586/matchfacts/vfl-bochum-vs-werder-bremen', 'https://www.fotmob.com/match/3903582/matchfacts/union-berlin-vs-bayern-m%C3%BCnchen']
['https://www.fotmob.com/match/3903585/matchfacts/wolfsburg-vs-1.-fc-k%C3%B6ln', 'https://www.fotmob.com/match/3900983/matchfacts/brentford-vs-leeds-united', 'https://www.fotmob.com/match/3900985/matchfacts/chelsea-vs-west-ham-united', 'https://www.fotmob.com/match/3900988/matchfacts/newcastle-united-vs-crystal-palace']
['https://www.fotmob.com/match/3900990/matchfacts/tottenham-hotspur-vs-fulha

['https://www.fotmob.com/match/4010293/matchfacts/aek-larnaca-vs-rennes', 'https://www.fotmob.com/match/4010294/matchfacts/fenerbah%C3%A7e-vs-dynamo-kyiv', 'https://www.fotmob.com/match/4010532/matchfacts/hjk-vs-real-betis', 'https://www.fotmob.com/match/4010533/matchfacts/ludogorets-razgrad-vs-roma']
['https://www.fotmob.com/match/4010661/matchfacts/malm%C3%B6-ff-vs-braga', 'https://www.fotmob.com/match/4010662/matchfacts/union-berlin-vs-union-st.gilloise', 'https://www.fotmob.com/match/4010194/matchfacts/fiorentina-vs-rfs', 'https://www.fotmob.com/match/4010193/matchfacts/hearts-vs-istanbul-ba%C5%9Fak%C5%9Fehir']
['https://www.fotmob.com/match/4010217/matchfacts/anderlecht-vs-silkeborg', 'https://www.fotmob.com/match/4010243/matchfacts/austria-wien-vs-hapoel-beer-sheva', 'https://www.fotmob.com/match/4010242/matchfacts/villarreal-vs-lech-poznan', 'https://www.fotmob.com/match/4010268/matchfacts/slovacko-vs-partizan-beograd']
['https://www.fotmob.com/match/4010580/matchfacts/ballkani-

In [105]:
open_file = open(f'scores.pkl', "wb")
pickle.dump(scores, open_file) 
open_file.close()

In [96]:
df['target'] = scores
df = df.drop(['url'], axis = 1)

In [81]:
accuracy_score(df['target'], df['odds_predict'])

0.6783216783216783

In [91]:
df

,Expected goals (xG),Total shots,Big chances,Big chances missed,Accurate passes,Accurate passes percentage,Fouls committed,Offsides,Corners,Shots off target,Shots on target,Blocked shots,Hit woodwork,Shots inside box,Shots outside box,xG open play,xG set play,xG on target (xGOT),Passes,Own half,Opposition half,Accurate long balls,Accurate long balls percentage,Accurate crosses,Accurate crosses percentage,Throws,Yellow cards,Red cards,Tackles won,Tackles won percentage,Interceptions,Blocks,Clearances,Keeper saves,Duels won,Ground duels won,Ground duels won percentage,Aerial duels won,Aerial duels won percentage,Successful dribbles,Successful dribbles percentage,score,posession,full_odds,xG penalty,odds_checker_sum,odds_predict,0,1,2,target
0,0.89,7.0,1.0,0.0,77.0,0.06,-3.0,1.0,1.0,4.0,0.0,3.0,0.0,7.0,0.0,0.97,-0.08,0.28,77.0,43.0,34.0,8.0,0.13,3.0,0.23,2.0,-2.0,0.0,-3.0,0.26,2.0,-3.0,-1.0,0.0,-2.0,-1.0,-0.03,-1.0,-0.10,4.0,-0.31,0.0,20.0,"[[2.4, 2.375, 2.3, 2.375, 2.3, 2.3, 2.2, 2.5, ...",NaN,1.091726,0,2.300000,2.387500,4.200000,0
1,-1.21,-5.0,-1.0,0.0,-54.0,-0.09,9.0,-1.0,-1.0,-1.0,-2.0,-2.0,0.0,-4.0,-1.0,-0.96,-0.26,-1.28,-49.0,5.0,-59.0,-7.0,-0.15,-1.0,0.00,-3.0,2.0,0.0,5.0,0.40,0.0,2.0,9.0,0.0,-3.0,-5.0,-0.16,2.0,0.06,-2.0,-0.11,-1.0,-16.0,"[[1.2857142857142856, 1.25, 1.2222222222222223...",NaN,1.087449,2,4.750000,13.000000,1.250000,2
2,-0.74,-3.0,-1.0,0.0,-140.0,-0.11,-3.0,0.0,-1.0,0.0,-1.0,-2.0,0.0,-1.0,-2.0,-0.06,-0.67,-1.25,-141.0,-31.0,-109.0,-7.0,-0.30,-1.0,-0.01,-5.0,-2.0,0.0,-2.0,0.43,4.0,1.0,5.0,-1.0,-15.0,-8.0,-0.40,-7.0,-0.64,-4.0,-0.80,-2.0,-26.0,"[[1.025, 1.0151515151515151, 1.02, 1.02, 1.02,...",NaN,1.044048,2,20.000000,56.000000,1.024390,2
3,0.37,4.0,1.0,1.0,56.0,0.04,-1.0,-1.0,-1.0,3.0,2.0,-1.0,1.0,2.0,2.0,0.37,NaN,0.97,56.0,-14.0,70.0,-4.0,-0.01,4.0,0.40,-2.0,-2.0,0.0,-3.0,-0.17,-4.0,1.0,-6.0,-3.0,-5.0,1.0,0.02,-6.0,-0.36,3.0,0.04,-1.0,12.0,"[[1.6153846153846154, 1.5714285714285714, 1.57...",NaN,1.064560,2,3.500000,6.500000,1.600000,1
4,-2.94,-12.0,-3.0,0.0,-242.0,-0.33,-1.0,-1.0,-4.0,0.0,-6.0,-6.0,1.0,-10.0,-2.0,-2.18,-0.76,-3.13,-234.0,-105.0,-137.0,-4.0,-0.31,-1.0,0.13,-8.0,1.0,0.0,1.0,0.07,3.0,6.0,13.0,2.0,7.0,5.0,0.24,2.0,0.10,3.0,0.60,-4.0,-50.0,"[[1.0002, 1.0909090909090908, 1.04, 1.0, 1.05,...",NaN,0.970938,2,226.000000,201.000000,1.040000,2
5,-0.95,1.0,-2.0,0.0,-21.0,-0.02,0.0,1.0,2.0,2.0,-1.0,0.0,0.0,0.0,1.0,-0.04,-0.13,-1.99,-19.0,-59.0,38.0,-1.0,0.20,0.0,-0.04,5.0,0.0,0.0,-4.0,-0.28,4.0,0.0,-4.0,0.0,-5.0,-6.0,-0.26,1.0,0.05,0.0,-0.25,-1.0,-4.0,"[[3.25, 3.0, 3.1, 3.1, 3.0, 3.1, 3.2, 3.0, 3.4...",-0.78,1.092484,2,3.126923,4.500000,1.816667,2
6,-0.33,-3.0,1.0,0.0,-104.0,-0.15,-1.0,3.0,-3.0,0.0,0.0,-3.0,0.0,1.0,-4.0,0.31,-0.64,0.71,-97.0,-51.0,-53.0,3.0,-0.09,2.0,0.29,-1.0,0.0,0.0,2.0,-0.06,-1.0,3.0,6.0,1.0,7.0,3.0,0.10,4.0,0.25,-1.0,0.11,1.0,-22.0,"[[1.4444444444444444, 1.4, 1.4, 1.444444444444...",NaN,1.086649,1,3.900000,1.444444,7.250000,1
7,-0.32,4.0,-1.0,-1.0,39.0,0.03,5.0,1.0,0.0,2.0,-1.0,3.0,0.0,1.0,3.0,-0.32,NaN,-0.74,36.0,-35.0,74.0,-16.0,-0.17,0.0,-0.04,3.0,0.0,0.0,4.0,0.00,5.0,-3.0,7.0,1.0,2.0,4.0,0.18,-2.0,-0.34,3.0,0.63,0.0,6.0,"[[2.05, 2.0, 2.05, 2.0, 2.1, 2.0, 2.05, 2.1, 2...",NaN,1.079192,1,2.500000,2.050000,5.225000,1
8,-0.04,-7.0,0.0,0.0,-76.0,-0.16,1.0,2.0,-2.0,-3.0,-2.0,-2.0,0.0,-7.0,0.0,-0.63,-0.19,0.36,-65.0,-8.0,-68.0,-4.0,-0.27,-1.0,0.06,-11.0,0.0,0.0,1.0,0.04,-1.0,3.0,5.0,1.0,4.0,2.0,0.06,2.0,0.16,3.0,0.05,-1.0,-20.0,"[[1.4444444444444444, 1.4444444444444444, 1.44...",0.78,1.085165,2,4.000000,7.000000,1.444444,2
9,-0.21,-6.0,-2.0,0.0,-110.0,-0.10,0.0,0.0,-3.0,-2.0,-4.0,0.0,-1.0,-1.0,-5.0,0.15,-0.36,-0.94,-102.0,-26.0,-84.0,-6.0,-0.11,0.0,0.20,3.0,0.0,0.0,-3.0,-0.25,-2.0,0.0,8.0,1.0,-4.0,-3.0,-0.14,-1.0,-0.14,0.0,0.34,-3.0,-20.0,"[[1.002, 1.002, 1.005, 1.005, 1.00333333333333...",NaN,1.009835,2,100.000000,251.000000,1.004167,2


In [92]:
today = f"{datetime.now():%Y%m%d}"
print(today, df.shape)
df.to_csv(f'data_{today}.csv')

20220911 (286, 51)


In [93]:
df

,Expected goals (xG),Total shots,Big chances,Big chances missed,Accurate passes,Accurate passes percentage,Fouls committed,Offsides,Corners,Shots off target,Shots on target,Blocked shots,Hit woodwork,Shots inside box,Shots outside box,xG open play,xG set play,xG on target (xGOT),Passes,Own half,Opposition half,Accurate long balls,Accurate long balls percentage,Accurate crosses,Accurate crosses percentage,Throws,Yellow cards,Red cards,Tackles won,Tackles won percentage,Interceptions,Blocks,Clearances,Keeper saves,Duels won,Ground duels won,Ground duels won percentage,Aerial duels won,Aerial duels won percentage,Successful dribbles,Successful dribbles percentage,score,posession,full_odds,xG penalty,odds_checker_sum,odds_predict,0,1,2,target
0,0.89,7.0,1.0,0.0,77.0,0.06,-3.0,1.0,1.0,4.0,0.0,3.0,0.0,7.0,0.0,0.97,-0.08,0.28,77.0,43.0,34.0,8.0,0.13,3.0,0.23,2.0,-2.0,0.0,-3.0,0.26,2.0,-3.0,-1.0,0.0,-2.0,-1.0,-0.03,-1.0,-0.10,4.0,-0.31,0.0,20.0,"[[2.4, 2.375, 2.3, 2.375, 2.3, 2.3, 2.2, 2.5, ...",NaN,1.091726,0,2.300000,2.387500,4.200000,0
1,-1.21,-5.0,-1.0,0.0,-54.0,-0.09,9.0,-1.0,-1.0,-1.0,-2.0,-2.0,0.0,-4.0,-1.0,-0.96,-0.26,-1.28,-49.0,5.0,-59.0,-7.0,-0.15,-1.0,0.00,-3.0,2.0,0.0,5.0,0.40,0.0,2.0,9.0,0.0,-3.0,-5.0,-0.16,2.0,0.06,-2.0,-0.11,-1.0,-16.0,"[[1.2857142857142856, 1.25, 1.2222222222222223...",NaN,1.087449,2,4.750000,13.000000,1.250000,2
2,-0.74,-3.0,-1.0,0.0,-140.0,-0.11,-3.0,0.0,-1.0,0.0,-1.0,-2.0,0.0,-1.0,-2.0,-0.06,-0.67,-1.25,-141.0,-31.0,-109.0,-7.0,-0.30,-1.0,-0.01,-5.0,-2.0,0.0,-2.0,0.43,4.0,1.0,5.0,-1.0,-15.0,-8.0,-0.40,-7.0,-0.64,-4.0,-0.80,-2.0,-26.0,"[[1.025, 1.0151515151515151, 1.02, 1.02, 1.02,...",NaN,1.044048,2,20.000000,56.000000,1.024390,2
3,0.37,4.0,1.0,1.0,56.0,0.04,-1.0,-1.0,-1.0,3.0,2.0,-1.0,1.0,2.0,2.0,0.37,NaN,0.97,56.0,-14.0,70.0,-4.0,-0.01,4.0,0.40,-2.0,-2.0,0.0,-3.0,-0.17,-4.0,1.0,-6.0,-3.0,-5.0,1.0,0.02,-6.0,-0.36,3.0,0.04,-1.0,12.0,"[[1.6153846153846154, 1.5714285714285714, 1.57...",NaN,1.064560,2,3.500000,6.500000,1.600000,1
4,-2.94,-12.0,-3.0,0.0,-242.0,-0.33,-1.0,-1.0,-4.0,0.0,-6.0,-6.0,1.0,-10.0,-2.0,-2.18,-0.76,-3.13,-234.0,-105.0,-137.0,-4.0,-0.31,-1.0,0.13,-8.0,1.0,0.0,1.0,0.07,3.0,6.0,13.0,2.0,7.0,5.0,0.24,2.0,0.10,3.0,0.60,-4.0,-50.0,"[[1.0002, 1.0909090909090908, 1.04, 1.0, 1.05,...",NaN,0.970938,2,226.000000,201.000000,1.040000,2
5,-0.95,1.0,-2.0,0.0,-21.0,-0.02,0.0,1.0,2.0,2.0,-1.0,0.0,0.0,0.0,1.0,-0.04,-0.13,-1.99,-19.0,-59.0,38.0,-1.0,0.20,0.0,-0.04,5.0,0.0,0.0,-4.0,-0.28,4.0,0.0,-4.0,0.0,-5.0,-6.0,-0.26,1.0,0.05,0.0,-0.25,-1.0,-4.0,"[[3.25, 3.0, 3.1, 3.1, 3.0, 3.1, 3.2, 3.0, 3.4...",-0.78,1.092484,2,3.126923,4.500000,1.816667,2
6,-0.33,-3.0,1.0,0.0,-104.0,-0.15,-1.0,3.0,-3.0,0.0,0.0,-3.0,0.0,1.0,-4.0,0.31,-0.64,0.71,-97.0,-51.0,-53.0,3.0,-0.09,2.0,0.29,-1.0,0.0,0.0,2.0,-0.06,-1.0,3.0,6.0,1.0,7.0,3.0,0.10,4.0,0.25,-1.0,0.11,1.0,-22.0,"[[1.4444444444444444, 1.4, 1.4, 1.444444444444...",NaN,1.086649,1,3.900000,1.444444,7.250000,1
7,-0.32,4.0,-1.0,-1.0,39.0,0.03,5.0,1.0,0.0,2.0,-1.0,3.0,0.0,1.0,3.0,-0.32,NaN,-0.74,36.0,-35.0,74.0,-16.0,-0.17,0.0,-0.04,3.0,0.0,0.0,4.0,0.00,5.0,-3.0,7.0,1.0,2.0,4.0,0.18,-2.0,-0.34,3.0,0.63,0.0,6.0,"[[2.05, 2.0, 2.05, 2.0, 2.1, 2.0, 2.05, 2.1, 2...",NaN,1.079192,1,2.500000,2.050000,5.225000,1
8,-0.04,-7.0,0.0,0.0,-76.0,-0.16,1.0,2.0,-2.0,-3.0,-2.0,-2.0,0.0,-7.0,0.0,-0.63,-0.19,0.36,-65.0,-8.0,-68.0,-4.0,-0.27,-1.0,0.06,-11.0,0.0,0.0,1.0,0.04,-1.0,3.0,5.0,1.0,4.0,2.0,0.06,2.0,0.16,3.0,0.05,-1.0,-20.0,"[[1.4444444444444444, 1.4444444444444444, 1.44...",0.78,1.085165,2,4.000000,7.000000,1.444444,2
9,-0.21,-6.0,-2.0,0.0,-110.0,-0.10,0.0,0.0,-3.0,-2.0,-4.0,0.0,-1.0,-1.0,-5.0,0.15,-0.36,-0.94,-102.0,-26.0,-84.0,-6.0,-0.11,0.0,0.20,3.0,0.0,0.0,-3.0,-0.25,-2.0,0.0,8.0,1.0,-4.0,-3.0,-0.14,-1.0,-0.14,0.0,0.34,-3.0,-20.0,"[[1.002, 1.002, 1.005, 1.005, 1.00333333333333...",NaN,1.009835,2,100.000000,251.000000,1.004167,2
